# Whitening transform
This notebook explores the effects of the *whitening transform*
(and its subroutine, the *Cholesky decomposition*)
on a few isolated example inputs.

### Cholesky decomposition
Decompose a matrix $M$ into two matrices $W$ of the same shape,
such that
$M = WW^{\top}$.

Note that $M$ must be
1. Symmetric
2. Positive definite
(In the example below, a covariance matrix is used.
These are always symmetric and positive definite.)

It is supposedly faster than the Eigenvalue-decomposition.
PyTorch implements both only on the CPU,
so I doubt either is particularly fast to use in Neural Networks.

In [2]:
import torch
import random
from rolim.tools.stats import sample_covar
from rolim.tools.testing import assert_tensor_eq

shape = (3, 10)
elements = [x for x in range(shape[0]*shape[1])]
random.shuffle(elements)
sample = torch.tensor(elements, dtype=torch.float).reshape(shape)
m = sample_covar(sample)
m


torch.Size([3])
tensor([[13.1000],
        [15.9000],
        [14.5000]])


tensor([[112.9889,  33.6778,  49.0556],
        [ 33.6778,  77.2111,  54.2778],
        [ 49.0556,  54.2778,  55.1667]])

In [5]:
w = torch.linalg.cholesky(m)
print(w)
print(w @ w.T)
torch.allclose(m, w@w.T)

tensor([[10.6296,  0.0000,  0.0000],
        [ 3.1683,  8.1959,  0.0000],
        [ 4.6150,  4.8385,  3.2338]])
tensor([[112.9889,  33.6778,  49.0556],
        [ 33.6778,  77.2111,  54.2778],
        [ 49.0556,  54.2778,  55.1667]])


True